In [1]:
from g4f.client import Client
import re
import pandas as pd

In [2]:
client = Client()

In [3]:
def generate_csv_report(user_input, csv_file_path):
    """
    從自然語言查詢生成 SQL 指令，並參考 CSV 檔案中的結構。

    Args:
        user_input (str): 用戶的自然語言查詢。
        csv_file_path (str): CSV 檔案的路徑。

    """

    # 讀取 CSV 檔案並將內容轉換為字串
    try:
        df = pd.read_csv(csv_file_path)
        csv_string = df.to_string(index=False)  # 不顯示索引
    except FileNotFoundError:
        print("找不到 CSV 檔案！請檢查檔案路徑是否正確。")
        return ""

    # 構建系統提示和用戶提示
    system_prompt = """
    1. 你是數據分析專家，能夠分析csv檔中的數據，並且給出分析報告(200字)。 
    """

    user_prompt = f"""
    1.參考此檔案並且給出分析報告:{csv_string}
    """

    # 使用 OpenAI API 生成報告
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input + user_prompt}
        ],
        temperature=0.1
    )
    csv_response = response.choices[0].message.content
    return csv_response


In [5]:
user_query ="此為上櫃股票個股本益比、殖利率、股價淨值比，請分析此數據"
csv_file = ".\BWIBBU_ALL.csv"
generate_csv_report(user_query,csv_file)

'根據提供的數據，我們可以對這些上櫃股票的本益比、殖利率和股價淨值比進行分析。首先，本益比（PE）方面，數據顯示市值較高的公司如南亞（77.75）和台化（108.33）有著極高的本益比，這可能意味著其市場預期增長較快，但同時也增加了投資風險。相對地，信大（10.32）和永裕（9.00）的本益比較低，顯示其相對被市場低估。\n\n在殖利率方面，環泥（7.08%）和達新（7.14%）提供了較高的收益回報，對於尋求穩定收入的投資者來說，這些股票具有吸引力。然而，像是東泥（0.97%）和味全（1.52%）的殖利率則偏低，可能不適合追求股息的策略。\n\n最後，股價淨值比（P/B）方面，股價淨值比大於1的股票，如大飲（4.91）和聯華食（5.81），可能顯示其市場價格高於內在價值，可能存在泡沫風險。總結來看，投資者應該根據自身的風險承受能力和投資期限，綜合考量這些指標來做出明智的投資決策。'